In [ ]:
 import os

# Path to your dataset in Google Drive
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/archive (4)/train_cancer"   # change if needed

folders = ["benign", "malignant"]

for folder in folders:
    folder_path = os.path.join(dataset_dir, folder)
    if os.path.exists(folder_path):
        count = len(os.listdir(folder_path))
        print(f"📁 {folder}: {count} images")
    else:
        print(f"⚠️ Folder not found: {folder_path}")


📁 benign: 320 images
📁 malignant: 320 images


In [ ]:
import os
import random

# Path to your dataset
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/archive (4)/train_cancer"  # change if needed

folders = ["benign", "malignant"]
keep_count = 320  # desired number

for folder in folders:
    folder_path = os.path.join(dataset_dir, folder)

    images = os.listdir(folder_path)

    # If more than 320, delete extras
    if len(images) > keep_count:
        extra = len(images) - keep_count
        delete_images = random.sample(images, extra)

        for img in delete_images:
            os.remove(os.path.join(folder_path, img))

        print(f"✔ Folder '{folder}' trimmed from {len(images)} to {keep_count} images")

    else:
        print(f"✔ Folder '{folder}' already has {len(images)} images (no need to trim)")


✔ Folder 'benign' trimmed from 330 to 320 images
✔ Folder 'malignant' trimmed from 324 to 320 images


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to your dataset folder in Drive
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/archive (4)/train_cancer"

# Output folder
output_dir = "/content/drive/MyDrive/Colab Notebooks/archive (4)"
os.makedirs(output_dir, exist_ok=True)

# Train/Test folders
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

for folder in ["train", "test"]:
    for cls in ["benign", "malignant"]:   # << lowercase
        os.makedirs(os.path.join(output_dir, folder, cls), exist_ok=True)

# Function to split each class
def split_and_copy(class_name):
    class_path = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_path)

    # Full paths for each image
    images = [os.path.join(class_path, img) for img in images]

    # 80-20 split
    train_files, test_files = train_test_split(images, test_size=0.2, random_state=42)

    # Copy train files
    for f in train_files:
        shutil.copy(f, os.path.join(train_dir, class_name))

    # Copy test files
    for f in test_files:
        shutil.copy(f, os.path.join(test_dir, class_name))

# Run splitting for both classes
split_and_copy("benign")      # << lowercase
split_and_copy("malignant")   # << lowercase

print("✅ Dataset split completed!")


✅ Dataset split completed!


In [ ]:
import os

output_dir = "/content/drive/MyDrive/Colab Notebooks/archive (4)"

folders = [
    "train/benign",
    "train/malignant",
    "test/benign",
    "test/malignant"
]

print("📊 Image Count After Splitting:\n")

for folder in folders:
    folder_path = os.path.join(output_dir, folder)
    print(f"{folder}: {len(os.listdir(folder_path))} images")


📊 Image Count After Splitting:

train/benign: 256 images
train/malignant: 256 images
test/benign: 64 images
test/malignant: 64 images


**CNN MODEL**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

BASE = "/content/drive/MyDrive/Colab Notebooks/archive (4)"  # update if needed

train_dir = os.path.join(BASE, "train")
test_dir  = os.path.join(BASE, "test")

IMG_SIZE = (224, 224)
BATCH_SIZE = 16

# Train loader with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
)

# Test loader (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Class mapping:", train_gen.class_indices)


Found 512 images belonging to 2 classes.
Found 128 images belonging to 2 classes.
Class mapping: {'benign': 0, 'malignant': 1}


In [ ]:
from tensorflow.keras import layers, models, optimizers

def build_cnn(input_shape=(224,224,3)):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

model = build_cnn()
model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,889 (429.25 KB)

 Trainable params: 109,889 (429.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_gen,
    epochs=20,
    validation_data=test_gen    # using test only, since you don't have val
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 0.5128 - loss: 0.6938 - val_accuracy: 0.6172 - val_loss: 0.6881
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 313ms/step - accuracy: 0.5539 - loss: 0.6888 - val_accuracy: 0.6562 - val_loss: 0.6842
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 305ms/step - accuracy: 0.5760 - loss: 0.6844 - val_accuracy: 0.6719 - val_loss: 0.6787
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 261ms/step - accuracy: 0.6052 - loss: 0.6813 - val_accuracy: 0.6875 - val_loss: 0.6705
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.6264 - loss: 0.6755 - val_accuracy: 0.7188 - val_loss: 0.6628
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 305ms/step - accuracy: 0.6516 - loss: 0.6703 - val_accuracy: 0.6953 - val_loss: 0.6524
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 269ms/step - accuracy: 0.6514 - loss: 0.6575 - val_accuracy: 0.6797 - val_loss: 0.6368
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 292ms/step - accuracy: 0.6583 - loss: 0.6480 - val_accuracy:

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/skin_cancer_scratch_model.keras")

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load your saved model
model = load_model("/content/drive/MyDrive/Colab Notebooks/skin_cancer_scratch_model.keras")

# Class mapping
idx_to_class = {0: "benign", 1: "malignant"}

def predict_image(img_path):
    # Load & preprocess
    img = Image.open(img_path).convert("RGB")
    img = img.resize((224, 224))

    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Raw sigmoid output → P(malignant)
    raw = model.predict(img_array)[0][0]

    # Convert based on mapping
    if raw < 0.5:
        class_label = "benign"        # class 0
        confidence = 1 - raw          # P(benign)
    else:
        class_label = "malignant"     # class 1
        confidence = raw              # P(malignant)

    return class_label, confidence, raw


In [ ]:
cls, conf, raw = predict_image("/content/drive/MyDrive/Colab Notebooks/archive (4)/train/malignant/10.jpg")

print("Predicted Class:", cls)
print("Confidence:", round(conf, 4))
print("Raw Sigmoid Output:", raw)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predicted Class: malignant
Confidence: 0.8697
Raw Sigmoid Output: 0.86966115
